# WIP: Calculation of free energy

This tutorial demonstrates the calculation of free energy using the ORCA python interface (OPI).

In this notebook, we will:
1. Import the required Python dependencies
2. Define a working directory
3. Prepare and visualize molecular structures
4. Set up and run ORCA calculations
5. Compute the free energy difference

## Step 1: Import Dependencies

We start by importing the modules needed for:
- Interfacing with ORCA input/output
- Plotting results
- Handling for directory

In [27]:
# > Import libraries for directory handling
from pathlib import Path

# > OPI imports for performing ORCA calculations and reading output
from opi.core import Calculator
from opi.output.core import Output
from opi.input.structures.structure import Structure
from opi.input.simple_keywords import BasisSet, Scf, Dft, SolvationModel, Solvent, Task

# > Import libraries for visualization
import py3Dmol
from IPython.display import display, HTML

## Step 2: Define Working directory

All actual calculations will be performed in a subfolder **RUN**.

In [28]:
# > Create a working directory for calculations if it doesn't already exist
working_dir = Path("RUN")
working_dir.mkdir(exist_ok=True)

## Step 3: Prepare and Visualize Input Structures

We define the XYZ atomic coordinates for the reactant and product molecules.


In [29]:
# > Dictionary of molecule names and their XYZ coordinates
xyz_data_list = {
    "reactant": """6

C    -0.374640     0.017356     0.209382
Br   -2.096602     0.059923     1.182216
H    -0.013725    -1.007560     0.247945
H     0.297020     0.702181     0.721417
H    -0.577587     0.336507    -0.810073
F     2.765535    -0.108405    -1.550886""",
    
    "product": """6

C     0.502797    -0.015722    -0.287744
Br   -2.842982     0.090180     1.619902
H     0.275582    -1.020463     0.077795
H     0.590517     0.684481     0.546964
H    -0.265180     0.319417    -0.989611
F     1.739267    -0.057892    -0.967306"""
}

# > Visualize the molecular structure using py3Dmol
html_views = []
for name, xyz_data in xyz_data_list.items():
    view = py3Dmol.view(width=300, height=300)
    view.addModel(xyz_data, "xyz")
    view.setStyle({'stick': {'radius': 0.1}, 'sphere': {'scale': 0.2}})

    view.setBackgroundColor('white')
    view.zoomTo()
    html = view._make_html()

    # > Save XYZ structure to file
    xyz_file_path = working_dir / f"struc_{name}.xyz"
    with open(xyz_file_path, "w") as f:
        f.write(xyz_data)

    html_div = f"""
    <div style="display:inline-block; text-align:center; margin:10px;">
        <h4>{name}</h4>
        {html}
    </div>
    """
    html_views.append(html_div)
full_html = "<div style='white-space: nowrap;'>" + "\n".join(html_views) + "</div>"
display(HTML(full_html))


## Step 4: Set Up and Run Calculations

We set up ORCA calculations using OPI:
- Use DFT with B3LYP functional and DEF2-TZVP basis set
- Apply CPCM solvation model with water
- Perform frequency calculations for thermodynamic data

In [30]:
def check_and_parse_output(output: Output)-> None:
    # > Check for proper termination of ORCA
    status = output.terminated_normally()
    if not status:
        # > ORCA did not terminate normally
        raise RuntimeError(f"ORCA did not terminate normally, see output file: {output.get_outfile()}")
    else:
        # > ORCA did terminate normally so we can parse the output
        output.parse()

    # Now check for convergence of the SCF
    if output.results_properties.geometries[0].single_point_data.converged:
        print("ORCA terminated normally and the SCF converged")
    else:
        raise RuntimeError("SCF DID NOT CONVERGE")

In [31]:
# > Set up path for structure
xyz_file = Path("./RUN/struc_reactant.xyz")

# > Create a Calculator object for ORCA input generation and execution
calc = Calculator(basename="reactant", working_dir=working_dir)

# > Load the molecular structure from XYZ file
structure = Structure.from_xyz(xyz_file)
calc.structure = structure
structure.multiplicity = 1
structure.charge = -1

# > Add calculation keywords
calc.input.add_simple_keywords(
      Scf.NOAUTOSTART,
      Dft.B3LYP,
      BasisSet.DEF2_TZVP,
      SolvationModel.CPCM(Solvent.WATER),
      Task.FREQ
)

# > Write the ORCA input file
calc.write_input()
# > Run the ORCA calculation
calc.run()
# > Get the output object
output_1 = calc.get_output()
check_and_parse_output(output_1)

ORCA terminated normally and the SCF converged


In [32]:
xyz_file = Path("./RUN/struc_product.xyz")
calc = Calculator(basename="product", working_dir=working_dir)
structure = Structure.from_xyz(xyz_file)
calc.structure = structure
structure.multiplicity = 1
structure.charge = -1
calc.input.add_simple_keywords(
      Scf.NOAUTOSTART,
      Dft.B3LYP,
      BasisSet.DEF2_TZVP,
      SolvationModel.CPCM(Solvent.WATER),
      Task.FREQ
)
calc.write_input()
calc.run()
output_2 = calc.get_output()
check_and_parse_output(output_2)

ORCA terminated normally and the SCF converged


## Step 5: Compute the Free Energy Difference
We extract the Gibbs free energy of each structure and compute the free energy difference (ΔG) between product and reactant.

In [33]:
# > Print the Gibbs free energie difference in kcal/mol
hartree_to_kcalmol = 627.509

freeEnergy_reactant = output_1.results_properties.geometries[0].thermochemistry_energies[0].freeenergyg * hartree_to_kcalmol
freeEnergy_product = output_2.results_properties.geometries[0].thermochemistry_energies[0].freeenergyg * hartree_to_kcalmol
delta_G = freeEnergy_product - freeEnergy_reactant

print("Gibbs Free Energy Difference:")
print(f"ΔG = {delta_G:.2f} kcal/mol")

Gibbs Free Energy Difference:
ΔG = -25.01 kcal/mol
